# Mobility in Puerto Rico following Hurricane Maria

### Baseline model: Measuring travel time from all origins (~1 mil WorldPop cells) to key health facilities
Baseline travel time is measured as the walking time (accounting for slope) from origin Point A to the closest node 
on the road network, plus driving time from there to the closest road node to a service.
Facilities are expected to be proximal to the road network, so no measure is taken between 
road and service.

Health facilities datasets:
- dialysis facilities
- hospitals
- pharmacies

### Storm disruption model: Accounting for storm-related disruptions to baseline travel time.
The disruption model simulates travel slowdowns and alternate routing due to geohazards, infrastructural damage, and facility incapacity. Disrupted travel times are measured by i) full road obstruction requiring alternate routing and ii) road speed penalty affecting total travel time. 

Storm-related hazards indicating complete obstruction:
- flooding intersection at 3 feet depth
- landslide intersection

Storm-related hazard indicating road speed penalty:
- flooding intersection at 1 and 2 feet depth

### 1. Configure script.

In [1]:
import os, sys
GISFolder = os.getcwd()

In [2]:
# Note: gostnet.py and associated files are in the current working directory.

os.getcwd()

'C:\\Users\\grace\\GIS\\puerto rico'

In [3]:
import GOSTnet as gn # Python couldn't find the module. Moved it into C:\Users\grace\Anaconda3\envs\access\Lib\site-packages
import pandas as pd
from geopandas import GeoDataFrame
import shapely
from shapely.geometry import Point, box
import geopandas as gpd
import osmnx as ox
import networkx as nx
import numpy as np
import rasterio as rt

In [4]:
# Didn't use in this iteration:
import fiona
import peartree
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [5]:
pth = os.path.join(GISFolder, "data\working files\gn")
pth

'C:\\Users\\grace\\GIS\\puerto rico\\data\\working files\\gn'

### 2. Get driving network for all islands in Puerto Rico. 

Reloading OSM data from file. Origins and health facilities are not yet incorporated. Travel measured in length (meters).

In [147]:
gTime = nx.read_gpickle("gTime.pickle")

#### Spatial join in Python on the gdf 

In [148]:
gTime_node_gdf = gn.node_gdf_from_graph(gTime)
gTime_edge_gdf = gn.edge_gdf_from_graph(gTime)
# Takes just a few seconds.

C:\Users\grace\Anaconda3\envs\access\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [149]:
gTime_edge_gdf = gTime_edge_gdf.drop(['oneway','lanes','ref','maxspeed','area', 'width','service','junction','access','tunnel','name', 'bridge'], axis=1)

In [150]:
gTime_edge_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 393582 entries, 0 to 393581
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   stnode    393582 non-null  int64   
 1   endnode   393582 non-null  int64   
 2   highway   393582 non-null  object  
 3   time      393582 non-null  float64 
 4   length    393582 non-null  float64 
 5   osmid     393582 non-null  object  
 6   mode      393582 non-null  object  
 7   geometry  393582 non-null  geometry
dtypes: float64(2), geometry(1), int64(2), object(3)
memory usage: 24.0+ MB


In [151]:
# This is the flood insurance zones dataset. Using the meters depth column (0,1,2, or 3 meters deep).
landslide = os.path.join(pth, "2017landslide_1km.shp")
landslide = gpd.read_file(landslide)

In [154]:
landslide.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 71431 entries, 0 to 71430
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Municipio  71431 non-null  object  
 1   Barrio     71431 non-null  object  
 2   geometry   71431 non-null  geometry
dtypes: geometry(1), object(2)
memory usage: 1.6+ MB


In [153]:
landslide = landslide[['Municipio', 'Barrio', 'geometry']]

In [155]:
landslide.crs

<Projected CRS: PROJCS["NAD83_2011_Puerto_Rico_and_Virgin_Is",GEOG ...>
Name: NAD83_2011_Puerto_Rico_and_Virgin_Is
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [156]:
gTime_edge_gdf.crs

<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [157]:
landslide = landslide.to_crs("EPSG:4326")

In [158]:
gTime_edge_gdf = gTime_edge_gdf.to_crs("EPSG:4326")

In [159]:
landslide.crs == gTime_edge_gdf.crs

True

In [160]:
join = gpd.sjoin(gTime_edge_gdf, landslide, how="left", op="intersects")
# Doesn't take long with the landslide dataset.

In [161]:
join.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 404926 entries, 0 to 393581
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   stnode       404926 non-null  int64   
 1   endnode      404926 non-null  int64   
 2   highway      404926 non-null  object  
 3   time         404926 non-null  float64 
 4   length       404926 non-null  float64 
 5   osmid        404926 non-null  object  
 6   mode         404926 non-null  object  
 7   geometry     404926 non-null  geometry
 8   index_right  20595 non-null   float64 
 9   Municipio    20595 non-null   object  
 10  Barrio       20595 non-null   object  
dtypes: float64(3), geometry(1), int64(2), object(5)
memory usage: 37.1+ MB


In [28]:
# nx.to_networkx_graph?

In [162]:
gTime_landslide_df = pd.DataFrame(join)

In [163]:
gTime_landslide_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404926 entries, 0 to 393581
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   stnode       404926 non-null  int64   
 1   endnode      404926 non-null  int64   
 2   highway      404926 non-null  object  
 3   time         404926 non-null  float64 
 4   length       404926 non-null  float64 
 5   osmid        404926 non-null  object  
 6   mode         404926 non-null  object  
 7   geometry     404926 non-null  geometry
 8   index_right  20595 non-null   float64 
 9   Municipio    20595 non-null   object  
 10  Barrio       20595 non-null   object  
dtypes: float64(3), geometry(1), int64(2), object(5)
memory usage: 37.1+ MB


In [164]:
gTime_landslide_df.to_csv(os.path.join(pth, 'gTime_landslide_1km.csv'))

#### Create speed penalties.
This is easier in R, but I want to avoid saving and reloading the file due to the pickiness of converting back to a graph.

In [165]:
gTime_landslide_df["penalty_l"] = 0

In [176]:
gTime_landslide_df.tail(40) # Time is in seconds.
# Not going to be nearly as many intersections as the flooding dataset. But when there is, the road is completely blocked.

,index,stnode,endnode,highway,time,length,osmid,mode,geometry,index_right,Municipio,Barrio,penalty_l
404886,393542,238026643,238082805,residential,7.67628,42.646,22138283,drive,"LINESTRING (-65.95805 18.38797, -65.95802 18.3...",-1.0,NaN,NaN,7.67628
404887,393543,238026643,238052057,residential,30.65508,170.306,22138283,drive,"LINESTRING (-65.95805 18.38797, -65.95813 18.3...",-1.0,NaN,NaN,30.65508
404888,393544,238026646,238026643,residential,24.45822,135.879,22128603,drive,"LINESTRING (-65.95678 18.38780, -65.95805 18.3...",-1.0,NaN,NaN,24.45822
404889,393545,5178916766,5178916767,residential,31.13190,172.955,22193591,drive,"LINESTRING (-67.14905 18.22829, -67.14888 18.2...",-1.0,NaN,NaN,31.13190
404890,393546,5178916767,5178916768,residential,11.20104,62.228,533892307,drive,"LINESTRING (-67.14801 18.22883, -67.14859 18.2...",-1.0,NaN,NaN,11.20104
404891,393547,5178916767,5178916766,residential,31.13190,172.955,22193591,drive,"LINESTRING (-67.14801 18.22883, -67.14801 18.2...",-1.0,NaN,NaN,31.13190
404892,393548,5178916767,5178916749,residential,11.36016,63.112,22193591,drive,"LINESTRING (-67.14801 18.22883, -67.14800 18.2...",-1.0,NaN,NaN,11.36016
404893,393549,5178916768,5178916767,residential,11.20104,62.228,533892307,drive,"LINESTRING (-67.14859 18.22891, -67.14801 18.2...",-1.0,NaN,NaN,11.20104
404894,393550,5178916774,5178916749,residential,11.74752,65.264,533892308,drive,"LINESTRING (-67.14778 18.22983, -67.14783 18.2...",-1.0,NaN,NaN,11.74752
404895,393551,238026666,238026671,residential,18.88704,104.928,22128605,drive,"LINESTRING (-66.07267 18.21207, -66.07168 18.2...",-1.0,NaN,NaN,18.88704


In [174]:
gTime_landslide_df.reset_index(inplace=True)

In [172]:
gTime_landslide_df["penalty_l"] = gTime_landslide_df['time'] 

In [54]:
gTime_landslide_df['index_right'] = gTime_landslide_df['index_right'].astype('float')

In [175]:
gTime_landslide_df[['index_right']] = gTime_landslide_df[['index_right']].fillna(value=-1) 
# Because it's complicated to use .loc (see below) on NaN values

In [177]:
gTime_landslide_df.loc[(gTime_landslide_df['index_right'] != -1), # Select rows where a landslide intersected.
       'penalty_l'] = (gTime_landslide_df['time'] * 3000) # Then make drive time impossibly long (for a 30 second baseline time, it would take 24+ hours)
# First time returned a ValueError: cannot reindex from a duplicate axis. Needed to do reset_index (see several lines up).

In [178]:
gTime_landslide_df.tail(40)
# Check that it changed row 404897 and 404915, the two rows of the last 40 that intersected a landslide.

,index,stnode,endnode,highway,time,length,osmid,mode,geometry,index_right,Municipio,Barrio,penalty_l
404886,393542,238026643,238082805,residential,7.67628,42.646,22138283,drive,"LINESTRING (-65.95805 18.38797, -65.95802 18.3...",-1.0,NaN,NaN,7.67628
404887,393543,238026643,238052057,residential,30.65508,170.306,22138283,drive,"LINESTRING (-65.95805 18.38797, -65.95813 18.3...",-1.0,NaN,NaN,30.65508
404888,393544,238026646,238026643,residential,24.45822,135.879,22128603,drive,"LINESTRING (-65.95678 18.38780, -65.95805 18.3...",-1.0,NaN,NaN,24.45822
404889,393545,5178916766,5178916767,residential,31.13190,172.955,22193591,drive,"LINESTRING (-67.14905 18.22829, -67.14888 18.2...",-1.0,NaN,NaN,31.13190
404890,393546,5178916767,5178916768,residential,11.20104,62.228,533892307,drive,"LINESTRING (-67.14801 18.22883, -67.14859 18.2...",-1.0,NaN,NaN,11.20104
404891,393547,5178916767,5178916766,residential,31.13190,172.955,22193591,drive,"LINESTRING (-67.14801 18.22883, -67.14801 18.2...",-1.0,NaN,NaN,31.13190
404892,393548,5178916767,5178916749,residential,11.36016,63.112,22193591,drive,"LINESTRING (-67.14801 18.22883, -67.14800 18.2...",-1.0,NaN,NaN,11.36016
404893,393549,5178916768,5178916767,residential,11.20104,62.228,533892307,drive,"LINESTRING (-67.14859 18.22891, -67.14801 18.2...",-1.0,NaN,NaN,11.20104
404894,393550,5178916774,5178916749,residential,11.74752,65.264,533892308,drive,"LINESTRING (-67.14778 18.22983, -67.14783 18.2...",-1.0,NaN,NaN,11.74752
404895,393551,238026666,238026671,residential,18.88704,104.928,22128605,drive,"LINESTRING (-66.07267 18.21207, -66.07168 18.2...",-1.0,NaN,NaN,18.88704


#### Convert back to graph

In [179]:
gTime2 = nx.from_pandas_edgelist(
    gTime_landslide_df,
    source="stnode",
    target="endnode",
    edge_attr=True,
)
# It ran. Only took a few seconds.

In [180]:
gn.example_edge(gTime2, 10)

(238026752, 238075156, {'index': 39981, 'highway': 'residential', 'time': 19.569059999999997, 'length': 108.717, 'osmid': 22137085, 'mode': 'drive', 'geometry': <shapely.geometry.linestring.LineString object at 0x000001E830477CC0>, 'index_right': -1.0, 'Municipio': nan, 'Barrio': nan, 'penalty_l': 19.569059999999997})
(238026752, 238026733, {'index': 393574, 'highway': 'residential', 'time': 34.03782, 'length': 189.099, 'osmid': 22128613, 'mode': 'drive', 'geometry': <shapely.geometry.linestring.LineString object at 0x000001E6BE47FAC8>, 'index_right': -1.0, 'Municipio': nan, 'Barrio': nan, 'penalty_l': 34.03782})
(238026752, 238072036, {'index': 37001, 'highway': 'residential', 'time': 30.338100000000004, 'length': 168.54500000000002, 'osmid': [22136924, 22137599], 'mode': 'drive', 'geometry': <shapely.geometry.linestring.LineString object at 0x000001E825328CF8>, 'index_right': -1.0, 'Municipio': nan, 'Barrio': nan, 'penalty_l': 30.338100000000004})
(238026752, 237995790, {'index': 365

In [181]:
gn.save(gTime2, 'gTime_landslide', pth, edges = False, nodes = False)

In [41]:
gn.save? # Tempting to make edges and nodes = True, but this returns an error. Better to save just the pickle file.

### 3. Origins and destinations

Measure distance from origin/destination to nearest node and save to file.

In [80]:
#%% If already created, load from file.
inOsnap = os.path.join(GISFolder, "data\working files\gn", "inOsnap.csv")
inOsnap = pd.read_csv(inOsnap)
inDsnap = os.path.join(GISFolder, "data\working files\gn", "inDsnap.csv")
inDsnap = pd.read_csv(inDsnap)
inHsnap = os.path.join(GISFolder, "data\working files\gn", "inHsnap.csv")
inHsnap = pd.read_csv(inHsnap)
inPsnap = os.path.join(GISFolder, "data\working files\gn", "inPsnap.csv")
inPsnap = pd.read_csv(inPsnap)

### Create flood-affected travel time values for the road nodes nearest to each service.

Using calculate_OD.

In [182]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
origins = list(inOsnap.NN.unique())

In [183]:
listD = list(inDsnap.NN.unique()) 
listH = list(inHsnap.NN.unique()) 
listP = list(inPsnap.NN.unique()) 
destslist = listD + listH + listP
dests = list(set(destslist))

In [184]:
len(origins) # 142,421 unique nearest nodes.

142421

In [185]:
len(dests) # 1,011 unique nearest nodes.

1011

In [186]:
len(listH)

69

In [187]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [188]:
gn.example_edge(gTime2)

(238026752, 238075156, {'index': 39981, 'highway': 'residential', 'time': 19.569059999999997, 'length': 108.717, 'osmid': 22137085, 'mode': 'drive', 'geometry': <shapely.geometry.linestring.LineString object at 0x000001E830477CC0>, 'index_right': -1.0, 'Municipio': nan, 'Barrio': nan, 'penalty_l': 19.569059999999997})


In [88]:
importlib.reload(gn)

<module 'GOSTnet' from 'C:\\Users\\grace\\Anaconda3\\envs\\access\\lib\\site-packages\\GOSTnet.py'>

In [ ]:
gn.calculate_OD?

In [189]:
OD_landslide = gn.calculate_OD(gTime2, origins, dests, fail_value, weight = 'penalty_l')
# Took maybe 10 min.

In [190]:
OD_landslide_df = pd.DataFrame(OD_landslide, index = origins, columns = dests)

In [191]:
OD_landslide_df.info()
# Created a 142421 x 1011 matrix. Same dimensions and size as the non-flood (1.1 GB)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142421 entries, 239148931 to 238880909
Columns: 1011 entries, 233324546 to 238196735
dtypes: float64(1011)
memory usage: 1.1 GB


In [193]:
OD_landslide_df.head()

,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,238549012,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
239148931,1.620300e+06,1.618829e+06,1.617235e+06,1.623279e+06,1.627657e+06,1.618892e+06,1.622829e+06,1.622483e+06,1.627642e+06,1.619158e+06,...,1.624693e+06,1.617288e+06,1.622714e+06,1.624124e+06,1.622942e+06,1.622847e+06,1.624322e+06,1.618799e+06,1.622442e+06,1.621031e+06
239077120,6.908073e+05,6.893366e+05,6.877420e+05,6.937869e+05,6.981641e+05,6.893999e+05,6.933362e+05,6.929905e+05,6.981495e+05,6.896657e+05,...,6.952007e+05,6.877952e+05,6.932216e+05,6.946315e+05,6.934495e+05,6.933550e+05,6.948292e+05,6.893066e+05,6.929497e+05,6.915381e+05
239127767,7.472560e+05,7.457853e+05,7.441908e+05,7.502356e+05,7.546128e+05,7.458486e+05,7.497850e+05,7.494392e+05,7.545983e+05,7.461144e+05,...,7.516494e+05,7.442440e+05,7.496704e+05,7.510802e+05,7.498982e+05,7.498038e+05,7.512779e+05,7.457553e+05,7.493984e+05,7.479868e+05
239136364,6.906559e+05,6.891852e+05,6.875907e+05,6.936355e+05,6.980127e+05,6.892485e+05,6.931849e+05,6.928391e+05,6.979982e+05,6.895143e+05,...,6.950493e+05,6.876439e+05,6.930703e+05,6.944801e+05,6.932981e+05,6.932037e+05,6.946778e+05,6.891552e+05,6.927983e+05,6.913867e+05
239077188,6.906876e+05,6.892169e+05,6.876223e+05,6.936672e+05,6.980444e+05,6.892802e+05,6.932165e+05,6.928707e+05,6.980298e+05,6.895460e+05,...,6.950810e+05,6.876755e+05,6.931019e+05,6.945118e+05,6.933297e+05,6.932353e+05,6.947095e+05,6.891869e+05,6.928300e+05,6.914184e+05


In [194]:
# Convert to minutes and save to file.
OD_landslide_min = OD_landslide_df[OD_landslide_df < fail_value] / 60
OD_landslide_min.to_csv(os.path.join(pth, 'OD_landslide.csv'))
# Takes a couple minutes.

In [195]:
OD_landslide_min.head(20)

,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,238549012,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
239148931,27004.996263,26980.484153,26953.908541,27054.655679,27127.609137,26981.539252,27047.145142,27041.382275,27127.366284,26985.969533,...,27078.218992,26954.795002,27045.235130,27068.732836,27049.032248,27047.458133,27072.027345,26979.984355,27040.702333,27017.175826
239077120,11513.454953,11488.942843,11462.367231,11563.114369,11636.067827,11489.997942,11555.603832,11549.840965,11635.824974,11494.428223,...,11586.677682,11463.253692,11553.693820,11577.191526,11557.490938,11555.916823,11580.486035,11488.443045,11549.161023,11525.634516
239127767,12454.267484,12429.755374,12403.179762,12503.926900,12576.880358,12430.810473,12496.416363,12490.653496,12576.637505,12435.240754,...,12527.490213,12404.066223,12494.506351,12518.004057,12498.303469,12496.729354,12521.298566,12429.255576,12489.973554,12466.447047
239136364,11510.932484,11486.420374,11459.844762,11560.591900,11633.545358,11487.475473,11553.081363,11547.318496,11633.302505,11491.905754,...,11584.155213,11460.731223,11551.171351,11574.669057,11554.968469,11553.394354,11577.963566,11485.920576,11546.638554,11523.112047
239077188,11511.459776,11486.947666,11460.372054,11561.119192,11634.072650,11488.002765,11553.608655,11547.845788,11633.829797,11492.433046,...,11584.682505,11461.258515,11551.698643,11575.196349,11555.495761,11553.921646,11578.490858,11486.447868,11547.165846,11523.639339
239176627,22592.044413,22567.532303,22540.956691,22641.703829,22714.657287,22568.587402,22634.193292,22628.430425,22714.414434,22573.017683,...,22665.267142,22541.843152,22632.283280,22655.780986,22636.080398,22634.506283,22659.075495,22567.032505,22627.750483,22604.223976
237676072,10839.244105,10907.812898,10872.947458,10844.420738,10840.160936,10907.905149,10837.844049,10842.241068,10705.523981,10913.298278,...,10823.985474,10872.164834,10840.643527,10815.345549,10852.787622,10845.498209,10866.559810,10908.312696,10855.762349,10879.714674
239149053,11508.992828,11484.480718,11457.905106,11558.652244,11631.605702,11485.535817,11551.141707,11545.378840,11631.362849,11489.966098,...,11582.215557,11458.791567,11549.231695,11572.729401,11553.028813,11551.454698,11576.023910,11483.980920,11544.698898,11521.172391
237686017,24352.416776,24327.904666,24301.329054,24402.076192,24475.029650,24328.959765,24394.565655,24388.802788,24474.786797,24333.390046,...,24425.639505,24302.215515,24392.655643,24416.153349,24396.452761,24394.878646,24419.447858,24327.404868,24388.122846,24364.596339
239158919,10839.252144,10907.820937,10872.955497,10844.428777,10840.168975,10907.913188,10837.852088,10842.249107,10705.532020,10913.306317,...,10823.993513,10872.172873,10840.651566,10815.353588,10852.795661,10845.506248,10866.567849,10908.320735,10855.770388,10879.722713


In [196]:
# Create POI-specific OD and save to file.
ODD_landslide = OD_landslide_df.loc[:, listD]
ODD_landslide = ODD_landslide[ODD_landslide < fail_value] / 60 
ODD_landslide.to_csv(os.path.join(pth, 'ODD_landslide.csv')) # Each takes a minute or so.

ODH_landslide = OD_landslide_df.loc[:, listH]
ODH_landslide = ODH_landslide[ODH_landslide < fail_value] / 60 
ODH_landslide.to_csv(os.path.join(pth, 'ODH_landslide.csv'))

ODP_landslide = OD_landslide_df.loc[:, listP]
ODP_landslide = ODP_landslide[ODP_landslide < fail_value] / 60 
ODP_landslide.to_csv(os.path.join(pth, 'ODP_landslide.csv'))

In [197]:
ODP_landslide.tail()

,238905285,238981133,2984408674,238960930,238067397,238363165,237993610,238174882,238556281,238468195,...,239020197,4198083801,243624306,237973577,237838093,237909639,499860461,239051712,238934844,238979708
238421388,76.802369,84.045622,88.206719,88.773683,94.891578,49.880754,61.313029,133.637212,238.934757,234.157621,...,135.233968,178.393583,69.952160,98.180580,172.293366,185.493366,84.828076,105.237524,97.994714,90.313594
238402093,76.840372,84.083625,88.244722,88.811686,94.929581,49.918757,61.351032,133.675215,238.972760,234.195624,...,135.271971,178.431586,69.990163,98.218583,172.331369,185.531369,84.866079,105.275527,98.032717,90.351597
238591938,91.244003,98.412563,102.573661,103.215317,109.333212,64.102621,75.444286,148.004153,253.301698,248.524562,...,89.708238,133.384544,66.152107,112.547522,186.660307,199.860307,99.266400,119.604466,112.406567,104.755228
238923898,96.648930,103.768677,107.929775,108.989472,115.061049,88.370370,74.778367,153.360267,258.657812,253.880676,...,36.675313,80.537460,108.825261,117.903636,192.016421,205.216421,133.205949,124.960580,117.762681,110.450993
238880909,99.652509,106.772256,110.933354,111.993051,118.064628,91.373949,77.781946,156.363846,261.661391,256.884255,...,37.810778,81.672925,111.828840,120.907215,195.020000,208.220000,136.209528,127.964159,120.766260,113.454572


### Compare flooding and baseline results.

In [198]:
# Run this even if already loaded. Name change for hazard file.
OD_base = os.path.join(pth, "OD.csv")
OD_base = pd.read_csv(OD_base)

In [199]:
OD_landslide = os.path.join(pth, "OD_landslide.csv")
OD_landslide = pd.read_csv(OD_landslide)

In [200]:
OD_base

,Unnamed: 0,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
0,239148931,114.824141,110.959791,65.620551,184.517029,218.033975,112.014890,177.794677,171.425579,85.948924,...,204.024088,66.515588,175.284588,195.431551,178.777999,177.272204,202.057806,110.459993,170.807127,147.563904
1,239077120,134.197077,107.545973,84.993487,181.103212,225.610682,108.601072,174.380859,168.011761,93.525632,...,205.775851,85.888524,171.870771,196.361803,175.364182,173.858386,198.643989,107.046175,167.393310,144.150087
2,239127767,131.989053,105.337949,82.785463,178.895188,223.402658,106.393048,172.172835,165.803737,91.317608,...,203.567827,83.680500,169.662747,194.153779,173.156158,171.650362,196.435965,104.838151,165.185286,141.942063
3,239136364,131.674608,105.023504,82.471018,178.580743,223.088213,106.078603,171.858390,165.489292,91.003163,...,203.253382,83.366055,169.348302,193.839334,172.841713,171.335917,196.121520,104.523706,164.870841,141.627618
4,239077188,132.201900,105.550796,82.998310,179.108035,223.615505,106.605895,172.385682,166.016584,91.530455,...,203.780674,83.893347,169.875594,194.366626,173.369005,171.863209,196.648812,105.050998,165.398133,142.154910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142416,238421388,274.348489,150.726605,262.410984,86.810998,39.038662,150.839943,80.654407,84.603840,172.531964,...,48.695262,261.493238,83.006299,57.307928,96.245695,87.874784,80.742644,151.216019,98.125120,122.596289
142417,238402093,274.386492,150.764608,262.448987,86.849001,39.076665,150.877946,80.692410,84.641843,172.569967,...,48.733265,261.531241,83.044302,57.345931,96.283698,87.912787,80.780647,151.254022,98.163123,122.634292
142418,238591938,263.079305,164.817016,247.437265,100.901409,22.135689,164.930354,94.744819,98.694252,127.237173,...,62.785673,248.332302,97.096710,71.398340,110.336106,101.965196,94.833056,165.306430,112.215532,136.686700
142419,238923898,209.664729,169.891442,194.022690,105.975835,91.021763,170.004780,99.819245,103.768677,73.822598,...,85.535147,194.917727,102.171136,76.942609,115.410532,107.039621,128.237466,170.380856,117.289957,138.778429


In [201]:
OD_landslide

,Unnamed: 0,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
0,239148931,27004.996263,26980.484153,26953.908541,27054.655679,27127.609137,26981.539252,27047.145142,27041.382275,27127.366284,...,27078.218992,26954.795002,27045.235130,27068.732836,27049.032248,27047.458133,27072.027345,26979.984355,27040.702333,27017.175826
1,239077120,11513.454953,11488.942843,11462.367231,11563.114369,11636.067827,11489.997942,11555.603832,11549.840965,11635.824974,...,11586.677682,11463.253692,11553.693820,11577.191526,11557.490938,11555.916823,11580.486035,11488.443045,11549.161023,11525.634516
2,239127767,12454.267484,12429.755374,12403.179762,12503.926900,12576.880358,12430.810473,12496.416363,12490.653496,12576.637505,...,12527.490213,12404.066223,12494.506351,12518.004057,12498.303469,12496.729354,12521.298566,12429.255576,12489.973554,12466.447047
3,239136364,11510.932484,11486.420374,11459.844762,11560.591900,11633.545358,11487.475473,11553.081363,11547.318496,11633.302505,...,11584.155213,11460.731223,11551.171351,11574.669057,11554.968469,11553.394354,11577.963566,11485.920576,11546.638554,11523.112047
4,239077188,11511.459776,11486.947666,11460.372054,11561.119192,11634.072650,11488.002765,11553.608655,11547.845788,11633.829797,...,11584.682505,11461.258515,11551.698643,11575.196349,11555.495761,11553.921646,11578.490858,11486.447868,11547.165846,11523.639339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142416,238421388,273.253033,149.617451,261.544444,85.827683,38.982082,149.709702,79.614876,84.045622,171.603982,...,48.287681,262.430905,82.059280,57.256400,94.203375,86.913963,80.372177,150.117249,97.566902,121.519228
142417,238402093,273.291036,149.655454,261.582447,85.865686,39.020085,149.747705,79.652879,84.083625,171.641985,...,48.325684,262.468908,82.097283,57.294403,94.241378,86.951966,80.410180,150.155252,97.604905,121.557231
142418,238591938,263.409318,163.984393,275.911385,100.269317,22.135689,164.076644,94.015543,98.412563,126.594943,...,62.733866,276.797846,96.500914,71.387657,108.645009,101.355596,94.810502,164.484191,111.933843,135.886169
142419,238923898,210.562234,169.340507,244.265586,105.948347,92.828745,169.432758,99.371657,103.768677,73.747858,...,85.513083,243.482962,102.171136,76.873158,114.315231,107.025818,128.087419,169.840305,117.289957,141.242283


In [202]:
OD_base == OD_landslide
# Returns True/False matrix

,Unnamed: 0,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142416,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
142417,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
142418,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
142419,True,False,False,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,True,False


In [203]:
((OD_base == OD_landslide).sum()).sum()
# Number of matching values. 

3434272

In [204]:
((OD_base == OD_landslide).sum()).sum() / ((OD_base == OD_base).sum()).sum()
# Percent of routes remaining the same after the hazard simulation.

0.023865219139646243

### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [205]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
ODD_landslide = os.path.join(pth, "ODD_landslide.csv")
ODD_landslide = pd.read_csv(ODD_landslide)
ODP_landslide = os.path.join(pth, "ODP_landslide.csv")
ODP_landslide = pd.read_csv(ODP_landslide)
ODH_landslide = os.path.join(pth, "ODH_landslide.csv")
ODH_landslide = pd.read_csv(ODH_landslide)

In [214]:
ODD_landslide.tail(10)

,NN,243545119,238600357,237883590,237875741,237992953,4377866427,238060667,553993016,239097861,...,238957886,2903017551,238975872,238997632,238965771,238065313,4155927397,239209720,5136150615,239344045
142411,238547482,235.351098,248.165838,171.022448,177.096912,75.453439,82.220353,82.981809,23.097679,50.656389,...,57.609310,58.071499,59.000253,55.370973,238.599409,77.168707,101.287820,113.507419,57.709642,237.512236
142412,238537241,240.023305,252.838044,175.694655,181.769118,80.125646,86.892069,87.654016,27.769885,55.328105,...,62.281026,62.743705,63.672459,60.043180,243.271616,81.840913,105.960026,118.179626,53.004924,242.184443
142413,5138198157,240.064138,252.878877,175.735488,181.809951,80.166479,86.932902,87.694849,27.810718,55.368938,...,62.321859,62.784538,63.713292,60.084013,243.312449,81.881746,106.000859,118.220459,52.686525,242.225276
142414,238521463,240.340107,253.154846,176.011457,182.085920,80.442448,87.208871,87.970818,28.086687,55.644907,...,62.597828,63.060507,63.989261,60.359982,243.588418,82.157715,106.276828,118.496428,51.743277,242.501245
142415,238217078,264.181473,276.996213,199.852823,205.927287,104.283814,99.687925,101.930786,51.928054,79.486273,...,86.439194,86.901874,87.830628,84.201348,267.429784,105.999082,130.118195,142.337794,52.010507,256.427183
142416,238421388,259.988192,272.802931,195.519116,201.593579,94.394497,56.474280,58.717142,82.859089,85.827683,...,79.077504,81.593099,82.521853,87.890640,263.096077,102.015253,125.784487,138.434455,71.631809,213.213538
142417,238402093,260.026195,272.840934,195.557119,201.631582,94.432500,56.512283,58.755145,82.897092,85.865686,...,79.115507,81.631102,82.559856,87.928643,263.134080,102.053256,125.822490,138.472458,71.669812,213.251541
142418,238591938,274.355133,272.611402,209.886057,215.960520,108.761438,70.605538,72.848399,97.297414,100.269317,...,93.519138,95.987641,96.963487,102.332274,277.463018,116.382194,140.151428,152.801396,90.647509,168.204499
142419,238923898,226.392459,219.764318,215.242171,221.316634,114.117552,73.636285,72.182481,131.236963,105.948347,...,99.198169,101.343755,102.367402,108.002496,243.547924,121.738308,145.507543,158.157510,165.934425,115.357415
142420,238880909,227.527924,220.899783,218.245750,224.320213,117.121131,76.639864,75.186060,134.240542,108.951926,...,102.201748,104.347334,105.370981,111.006075,244.683389,124.741887,148.511122,161.161089,168.938004,116.492880


In [213]:
ODD_landslide.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) # Repeat for each OD set, if needed.

#### Find first nearest POI for each origin node. Run this block for each variable.

In [215]:
ODD_landslide["1D"] = 0
sub = ODD_landslide.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
ODD_landslide["1D"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
D1_landslide = ODD_landslide[['NN', '1D']] # Remove unnecessary OD values.
D1_landslide.to_csv(os.path.join(pth, '1D_landslide.csv'))

In [217]:
D1_landslide.tail(40) 

,NN,1D
142381,3422672432,18.638312
142382,233331747,26.318761
142383,233370841,23.569514
142384,233363808,26.201311
142385,233324358,30.779051
142386,5125256148,43.283626
142387,238797934,42.141003
142388,238672387,4.420182
142389,5186570054,30.226955
142390,238256805,16.589999


In [218]:
ODH_landslide["1H"] = 0
sub = ODH_landslide.iloc[:,1:-1] # Filtering out the newly created field and the node ID column.
ODH_landslide["1H"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
H1_landslide = ODH_landslide[['NN', '1H']] # Remove unnecessary OD values.
H1_landslide.to_csv(os.path.join(pth, '1H_landslide.csv'))

In [219]:
H1_landslide.head(10)

,NN,1H
0,239148931,26941.704312
1,239077120,11450.163002
2,239127767,12390.975533
3,239136364,11447.640533
4,239077188,11448.167825
5,239176627,22528.752462
6,237676072,10702.819166
7,239149053,11445.700877
8,237686017,24289.124825
9,239158919,10702.827205


In [220]:
ODP_landslide["1P"] = 0
sub = ODP_landslide.iloc[:,1:-1] # Filtering out the newly created field and the node ID column.
ODP_landslide["1P"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
P1_landslide = ODP_landslide[['NN', '1P']] # Remove unnecessary OD values.
P1_landslide.to_csv(os.path.join(pth, '1P_landslide.csv'))

In [221]:
P1_landslide.head(10)

,NN,1P
0,239148931,22482.683402
1,239077120,4634.970348
2,239127767,5575.782879
3,239136364,4632.447879
4,239077188,4632.975171
5,239176627,18069.731552
6,237676072,10665.960198
7,239149053,4630.508223
8,237686017,17473.932171
9,239158919,10665.968236


#### Compare to baseline values.

In [222]:
D1_base = os.path.join(pth, "1D.csv")
D1_base = pd.read_csv(D1_base)
P1_base = os.path.join(pth, "1P.csv")
P1_base = pd.read_csv(P1_base)
H1_base = os.path.join(pth, "1H.csv")
H1_base = pd.read_csv(H1_base)

In [225]:
P1_base.head()

,NN,1P
0,239148931,14.262696
1,239077120,19.665247
2,239127767,17.457223
3,239136364,17.142778
4,239077188,17.670070


In [224]:
P1_base = P1_base.loc[:,['NN', '1P']] # Remove unnecessary Unnamed: 0 column.
D1_base = D1_base.loc[:,['NN', '1D']]
H1_base = H1_base.loc[:,['NN', '1H']]

In [226]:
P1_landslide.head()

,NN,1P
0,239148931,22482.683402
1,239077120,4634.970348
2,239127767,5575.782879
3,239136364,4632.447879
4,239077188,4632.975171


In [227]:
P1_base == P1_landslide

,NN,1P
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
142416,True,False
142417,True,False
142418,True,True
142419,True,False


In [228]:
((P1_base['1P'] == P1_landslide['1P']).sum()).sum()
# Number of routes that remained the same.

67403

In [229]:
((P1_base['1P'] == P1_landslide['1P']).sum()).sum() / ((P1_base['1P'] == P1_base['1P']).sum()).sum()
# 47.4 percent of routes remained the same after the hazard simulation.

0.4735052582034296

In [230]:
((D1_base['1D'] == D1_landslide['1D']).sum()).sum()

30787

In [231]:
((D1_base['1D'] == D1_landslide['1D']).sum()).sum() / ((D1_base['1D'] == D1_base['1D']).sum()).sum()
# 21.6 percent of routes remained the same after the hazard simulation.

0.2162950160884655

In [232]:
((H1_base['1H'] == H1_landslide['1H']).sum()).sum()

26694

In [233]:
((H1_base['1H'] == H1_landslide['1H']).sum()).sum() / ((H1_base['1H'] == H1_base['1H']).sum()).sum()
# 18.8 percent of routes remained the same after the hazard simulation.

0.18753951861063103

### Create multi-modal travel times by combining walk time to road with drive time to nth nearest service.

In [234]:
# If starting new session, re-load from disk.
zwalk = os.path.join(pth, "zwalk.csv") 
zwalk = pd.read_csv(zwalk)

In [235]:
D1_landslide = os.path.join(pth, "1D_landslide.csv")
D1_landslide = pd.read_csv(D1_landslide)

In [236]:
zwalk.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183


In [237]:
D1_landslide.head()

,Unnamed: 0,NN,1D
0,0,239148931,26922.071692
1,1,239077120,11430.530382
2,2,239127767,12371.342913
3,3,239136364,11428.007913
4,4,239077188,11428.535205


In [238]:
# Merge nearest POIs and walktimes
zwalkD_landslide = zwalk.merge(D1_landslide, how='left', left_on='NN', right_on='NN', sort=False)
zwalkD_landslide.head()

,Unnamed: 0_x,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,Unnamed: 0_y,1D
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,0.0,26922.071692
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,0.0,26922.071692
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,0.0,26922.071692
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,0.0,26922.071692
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,1.0,11430.530382


In [239]:
# Merge nearest POIs and walktimes
zwalkP_landslide = zwalk.merge(P1_landslide, how='left', left_on='NN', right_on='NN', sort=False)
zwalkP_landslide.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1P
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,22482.683402
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,22482.683402
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,22482.683402
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,22482.683402
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,4634.970348


In [240]:
# Merge nearest POIs and walktimes
zwalkH_landslide = zwalk.merge(H1_landslide, how='left', left_on='NN', right_on='NN', sort=False)
zwalkH_landslide.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1H
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,26941.704312
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,26941.704312
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,26941.704312
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,26941.704312
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,11450.163002


In [241]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkH_landslide["mm1H"] = 0
zwalkH_landslide["mm1H"] = zwalkH_landslide["walk_time"] + zwalkH_landslide["1H"]

In [243]:
zwalkH_landslide.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1H,mm1H
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,26941.704312,26951.702917
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,26941.704312,26949.965266
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,26941.704312,26948.994800
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,26941.704312,26949.484253
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,11450.163002,11463.134185


In [244]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkP_landslide["mm1P"] = 0
zwalkP_landslide["mm1P"] = zwalkP_landslide["walk_time"] + zwalkP_landslide["1P"]

In [245]:
zwalkP_landslide.tail()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1P,mm1P
1076135,1076135,0.888915,NaN,1075779,Salinas,238881983,238881983,5.041526,43.878211,17.351342,61.229553
1076136,1076136,0.889920,NaN,1075780,Salinas,238881983,238881983,5.032043,44.713233,17.351342,62.064575
1076137,1076137,0.900318,NaN,1075781,Salinas,238859820,238859820,5.027474,45.359462,8.687345,54.046807
1076138,1076138,0.894649,NaN,1075782,Salinas,238859820,238859820,5.032116,45.417854,8.687345,54.105199
1076139,1076139,0.858682,NaN,1075783,Salinas,238289242,238289242,5.022637,44.728867,25.605756,70.334623


In [246]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkD_landslide["mm1D"] = 0
zwalkD_landslide["mm1D"] = zwalkD_landslide["walk_time"] + zwalkD_landslide["1D"]

In [250]:
zwalkD_landslide.tail()

,wpop,xmid,wid,municipio,NN,walk_time,1D,mm1D
1076135,0.888915,NaN,1075779,Salinas,238881983,43.878211,41.235351,85.113561
1076136,0.889920,NaN,1075780,Salinas,238881983,44.713233,41.235351,85.948584
1076137,0.900318,NaN,1075781,Salinas,238859820,45.359462,31.589310,76.948771
1076138,0.894649,NaN,1075782,Salinas,238859820,45.417854,31.589310,77.007163
1076139,0.858682,NaN,1075783,Salinas,238289242,44.728867,24.840352,69.569219


In [248]:
zwalkD_landslide = zwalkD_landslide[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1D', 'mm1D']]
zwalkH_landslide = zwalkH_landslide[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1H', 'mm1H']]
zwalkP_landslide = zwalkP_landslide[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1P', 'mm1P']]

In [249]:
zwalkH_landslide.to_csv(os.path.join(pth, 'H_mm_landslide.csv'))
zwalkP_landslide.to_csv(os.path.join(pth, 'P_mm_landslide.csv'))
zwalkD_landslide.to_csv(os.path.join(pth, 'D_mm_landslide.csv'))